# Load Data 

In [ ]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
from os import listdir
import numpy as np
import pandas as pd
import re
from difflib import SequenceMatcher

In [ ]:
# please alter the path to locate the file you downloaded
# files used are from "training-RiskFactors-Complete-Set1" and "training-RiskFactors-Complete-Set2" folder
trainingpath1 = 'E:/JHU/课程/datadesign/NLP/modified_tagged_files/training-RiskFactors-Complete-Set1'
trainingpath2 = 'E:/JHU/课程/datadesign/NLP/modified_tagged_files/training-RiskFactors-Complete-Set2'
testingpath = "E:/JHU/课程/datadesign/NLP/modified_tagged_files/testing-RiskFactors-Complete"
trainingpath = [trainingpath1, trainingpath2] 

# collect all file names to read in later
names_1 = [f for f in listdir(trainingpath1) if f.endswith('.xml')]
names_2 = [f for f in listdir(trainingpath2) if f.endswith('.xml')]
names_3 = [f for f in listdir(testingpath) if f.endswith('.xml')]

In [ ]:
def read_xml(path,file):
    tree = ET.parse(path+'/'+file)
    root = tree.getroot()
    text = root.find('TEXT').text
    return text

### Make raw input for GUI

In [ ]:
# create empty dataframe to save corpus used as input in GUI
corpus = pd.DataFrame(columns=['PatientID','TimeID','RawText','TagCategory','Rawtag','CleanedText','AnnotatedText','Tag_doc','Real_text_tag'])
corpus_test = pd.DataFrame(columns=['PatientID','TimeID','RawText','TagCategory','Rawtag','CleanedText','AnnotatedText','Real_text_tag','Pred_text_tag','Real_tag_doc','Pred_tag_doc'])

In [ ]:
# if path is a list of paths
patientID = []
timeID = []
rawText = []
tagCategories = []
rawtag = []
annotatedText = []
tag_text = []

for path in trainingpath:
    corpus_eachpath = pd.DataFrame(columns=['PatientID','TimeID','RawText','TagCategory','Rawtag','CleanedText','AnnotatedText','Tag_doc','Real_text_tag'])
    for file in os.listdir(path):
        #read the text part of xml file
        text = read_xml(path,file)
        #print(text)

        #save the PatientID, TimeID to the first two columns
        patientID.append(file.split('-')[0])
        timeID.append(file.split('-')[1].split('.')[0])
        #save the text to the third column
        rawText.append(text)

        #read the tags part of xml file
        tree = ET.parse(path+'/'+file)
        root = tree.getroot()
        tags = root.findall('TAGS') 
        #save the tags to the fourth and fifth column
        for tag in tags: #<TAGS>
        
            tagcategory_eachfile = []
            rawtag_attrib_eachfile =[]
            tag_text_eachfile = []

            for child in tag:
                #print(child.tag)
                if child.tag != 'PHI':
                    tagcategory_eachfile.append(child.tag) #save the tag category of each file to the a list
                    for subchild in child:
                        #combine the subchild tag  and subchild attrib to a string
                        subchild_attrib = subchild.tag + ' ' + str(subchild.attrib)
                        #print(subchild_attrib)
                        rawtag_attrib_eachfile.append(subchild_attrib)
                    
                        if subchild.attrib.keys().__contains__('text'):
                            subchild_text = subchild.attrib['text']
                            subchild_text = subchild_text.strip()#remove the blank space in the text
                            #print(subchild_text)
                            subchild_tag = subchild.tag
        
                            tag_text_eachfile.append((subchild_text,subchild_tag))

        
            tagCategories.append(tagcategory_eachfile)           
            rawtag.append(rawtag_attrib_eachfile)
            annotatedText.append(tag_text_eachfile)

    corpus_eachpath['PatientID'] = patientID
    corpus_eachpath['TimeID'] = timeID
    corpus_eachpath['RawText'] = rawText
    #corpus_eachpath['TagCategory'] = tagCategories
    corpus_eachpath['Rawtag'] = rawtag
    #corpus_eachpath['AnnotatedText'] = annotatedText
    #print(corpus)
    
    corpus = pd.concat([corpus_eachpath],axis=0)

'''
#convert the list in the column 'Rawtag_text' to string and between each element in the list, add a enter
for index,entry in enumerate(corpus['Rawtag_text']):
    corpus['Rawtag_text'][index] = '//'.join(entry)
'''
print(corpus['Rawtag'])

In [ ]:
patientID = []
timeID = []
rawText = []
tagCategories = []
rawtag = []
annotatedText = []
tag_text = []


for file in os.listdir(testingpath):
    #read the text part of xml file
    text = read_xml(testingpath,file)
    #print(text)

    #save the PatientID, TimeID to the first two columns
    patientID.append(file.split('-')[0])
    timeID.append(file.split('-')[1].split('.')[0])
    #save the text to the third column
    rawText.append(text)

    #read the tags part of xml file
    tree = ET.parse(testingpath+'/'+file)
    root = tree.getroot()
    tags = root.findall('TAGS') 
    #save the tags to the fourth and fifth column
    for tag in tags: #<TAGS>
        
        tagcategory_eachfile = []
        rawtag_attrib_eachfile =[]
        tag_text_eachfile = []

        for child in tag:
            #print(child.tag)
            if child.tag != 'PHI':
                tagcategory_eachfile.append(child.tag) #save the tag category of each file to the a list
                for subchild in child:
                    #combine the subchild tag  and subchild attrib to a string
                    subchild_attrib = subchild.tag + ' ' + str(subchild.attrib)
                    #print(subchild_attrib)
                    rawtag_attrib_eachfile.append(subchild_attrib)
                    
                    if subchild.attrib.keys().__contains__('text'):
                        subchild_text = subchild.attrib['text']
                        subchild_text = subchild_text.strip()#remove the blank space in the text
                        #print(subchild_text)
                        subchild_tag = subchild.tag
        
                        tag_text_eachfile.append((subchild_text,subchild_tag))

        
        tagCategories.append(tagcategory_eachfile)           
        rawtag.append(rawtag_attrib_eachfile)
        annotatedText.append(tag_text_eachfile)

corpus_test['PatientID'] = patientID
corpus_test['TimeID'] = timeID
corpus_test['RawText'] = rawText
#corpus_test['TagCategory'] = tagCategories
corpus_test['Rawtag'] = rawtag
#corpus_test['AnnotatedText'] = annotatedText
#print(corpus)

'''
#convert the list in the column 'Rawtag_text' to string and between each element in the list, add a enter
for index,entry in enumerate(corpus['Rawtag_text']):
    corpus['Rawtag_text'][index] = '//'.join(entry)
'''
print(corpus_test['RawText'])

### Make input for machine learning

In [ ]:
# an empty dataframe for machine learning input
all1_df = pd.DataFrame(np.zeros((len(names_1), 3), dtype=object), columns=['text', 'annotation','loc'])
all2_df = pd.DataFrame(np.zeros((len(names_2), 3), dtype=object), columns=['text', 'annotation','loc'])
test_df = pd.DataFrame(np.zeros((len(names_3), 3), dtype=object), columns=['text', 'annotation','loc'])

In [ ]:
def to_df(names,df, file_path,PHI_status = True, type = 'train'):
    # Get text and reformat it
    n = 0
    for name in names:
        tree = ET.parse(file_path +'/'+ name)
        root = tree.getroot()
        nt = re.sub('\n',' ',root[0].text)
        nt = re.sub('\t',' ',nt) 
        nt = re.sub('"',"'",nt)
        ## sample 214 has a weird character
        nt = re.sub('>','&gt;',nt) 
        nt = re.sub('<','&lt;',nt)
        ## new wired character
        nt = re.sub('Â','',nt)
        nt = re.sub('â','',nt)
        nt = re.sub('€','',nt)
        nt = re.sub('™','',nt)
        df['text'][n] = nt
        n+=1
    
    # Get annotations
    n = 0
    for name in names:
        tree = ET.parse(file_path +'/'+ name)
        root = tree.getroot()
        ## Get the labels
        tag_list = []
        loc_list = []
        # get PHI labels if based on condition
        if PHI_status == True:
            PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
            for k in range(PHI,len(root[1])):
                tag_list.append((root[1][k].attrib['text'],root[1][k].tag))
        
        # get the rest of labels
        for k in range(len(root[1])):
            #if root[1][k].tag == 'SMOKER':
            #    continue
            if type == 'train':
                if root[1][k].tag == 'FAMILY_HIST':
                    continue
            
            for m in range(len(root[1][k])):
                if root[1][k][m].attrib.keys().__contains__('text') == False:
                    continue
                tag_list.append((root[1][k][m].attrib['text'],root[1][k][m].tag))
                loc_list.append((root[1][k][m].attrib['start'],root[1][k][m].attrib['end']))
        df['annotation'][n] = tag_list
        df['loc'][n] = loc_list
        n+=1
    return df

all_1 = to_df(names_1,all1_df, trainingpath1, PHI_status = False, type = 'train')
all_2 = to_df(names_2,all2_df, trainingpath2, PHI_status = False, type = 'train')
test_df = to_df(names_3,test_df, testingpath, PHI_status = False, type='test')

train_df = pd.concat([all_1, all_2], ignore_index=True)

--------------------------------------------------------------------------------------Next section-----------------------------------------------------------------------------------------

# Data pre-processing

### define function

In [ ]:
## define functions to merge partial overlapped text
## example: "I like apple" + "apple that is red" --> "I like apple that is red"
## ls1 needs to be at the left of ls2 in the text
def intersection(ls1, ls2):
    index = [i for i, x in enumerate([x == ls2[0] for x in ls1]) if x]
    if index[-1] == len(ls1)-1:
        index = index[:-1]
    cut = 10000
    eva_temp = 10000
    for i in index:
        if (len(ls1) -i) < len(ls2):
            l = len(ls1) -i
        else:
            l = len(ls2)
        eva = sum([ls1[x+i] != ls2[x] for x in range(l)])
        if eva < eva_temp:
            cut = i
            eva_temp = eva
    out = "".join(ls1[0:cut]+ls2)
    return out

## define pre-process function
## preprocess tagged text and location
def pre_process(input):
    df = input.copy()
    for i in range(df.shape[0]):  
        for j in range(len(df['annotation'][i])):
                ## remove extra spaces in the beginning and end of the annotation
                if re.search("^ +.*",df['annotation'][i][j][0]) != None or re.search(".* +$",df['annotation'][i][j][0]) != None:
                    front = len(df['annotation'][i][j][0]) - len(re.sub("^ +","",df['annotation'][i][j][0]))
                    end = len(df['annotation'][i][j][0]) - len(re.sub(" +$","",df['annotation'][i][j][0]))
                    df['loc'][i][j] = (str(int(df['loc'][i][j][0])+front),str(int(df['loc'][i][j][1])-end))
                    df['annotation'][i][j] = (re.sub(" +$","", re.sub("^ +","",df['annotation'][i][j][0])),df['annotation'][i][j][1])
                ## fix "Record" in "Record date" taged as SMOKER
                if int(df['loc'][i][j][0]) == df['text'][i].find('Record'): 
                    df['loc'][i][j] = ('','')
                    df['annotation'][i][j] = ('','')
                    
    return df

## main function to remove duplicated tags
## example: 1) "I like apple that is red" & "I like apple"; 2) "I like apple" + "apple that is red"
def rm_dup(input):
    df = input.copy()
    symbol = [",",".","-"]
    for i in range(df.shape[0]):
        for j in range(len(df['annotation'][i])):
            ## used k starts at j+1 to avoid self compare
            for k in np.arange(j+1,len(df['annotation'][i])):
                ## move on if the compared lables are ("","") due to ealier process
                if df['loc'][i][j] == ('',''):
                    break
                if df['loc'][i][k] == ('',''):
                    continue

                ## find location contained within each other ("I like apple that is red" & "I like apple")
                if int(df['loc'][i][j][0]) >= int(df['loc'][i][k][0]) and int(df['loc'][i][j][1]) <= int(df['loc'][i][k][1]):
                    df['loc'][i][j] = ('','')
                    df['annotation'][i][j] = ('','')
                    continue
                
                elif int(df['loc'][i][j][0]) <= int(df['loc'][i][k][0]) and int(df['loc'][i][j][1]) >= int(df['loc'][i][k][1]):
                    df['loc'][i][k] = ('','')
                    df['annotation'][i][k] = ('','')
                    continue

                ## find location that overlap ("I like apple" + "apple that is red")
                if int(df['loc'][i][j][0]) < int(df['loc'][i][k][0]) and int(df['loc'][i][j][1]) < int(df['loc'][i][k][1]) and int(df['loc'][i][j][1]) > int(df['loc'][i][k][0]):
                    new_s = intersection(df['annotation'][i][j][0],df['annotation'][i][k][0])
                    df['annotation'][i][j] = (new_s,df['annotation'][i][j][1])
                    df['loc'][i][j] = (df['loc'][i][j][0],df['loc'][i][k][1])
                    df['loc'][i][k] = ('','')
                    df['annotation'][i][k] = ('','')
                    continue
            
                elif int(df['loc'][i][j][0]) > int(df['loc'][i][k][0]) and int(df['loc'][i][j][1]) > int(df['loc'][i][k][1]) and int(df['loc'][i][j][0]) < int(df['loc'][i][k][1]):
                    #print(i,j,k)
                    #the order here matters, the first one should be the left most
                    new_s = intersection(df['annotation'][i][k][0],df['annotation'][i][j][0])
                    df['annotation'][i][k] = (new_s,df['annotation'][i][k][1])
                    df['loc'][i][k] = (df['loc'][i][k][0],df['loc'][i][j][1])
                    df['loc'][i][j] = ('','')
                    df['annotation'][i][j] = ('','')
                    continue

    return df

### preprocess train and test

In [ ]:
## preprocess the data
train_df = pre_process(train_df)
train_n = rm_dup(train_df)
train_n = rm_dup(train_n)

test_df = pre_process(test_df)
test_n = rm_dup(test_df)
test_n = rm_dup(test_n)

In [ ]:
## remove all the ("","")
for j in range(train_n.shape[0]):
    train_n['annotation'][j] = [train_n['annotation'][j][i] for i in range(len(train_n['annotation'][j])) if train_n['annotation'][j][i][0] != '']
    train_n['loc'][j] = [train_n['loc'][j][i] for i in range(len(train_n['loc'][j])) if train_n['loc'][j][i][0] != '']
    ## order the tags by their starting position to avoid issue with finding text
    ## example: "I like red apple that is red" {("red",pos1), ("red apple", pos2)} --> "I like XXX apple that is red" {("red apple", pos2)}
    ## This issue originates how to match predicted back to original text after all the preprocessing where location info doesn't match
    start = np.array([int(train_n['loc'][j][i][0]) for i in range(len(train_n['loc'][j]))])
    order = start.argsort()
    train_n['annotation'][j] = [train_n['annotation'][j][i] for i in order]
    train_n['loc'][j] = [train_n['loc'][j][i] for i in order]
train = train_n.drop('loc', axis=1)
train.head()

for j in range(test_n.shape[0]):
    test_n['annotation'][j] = [test_n['annotation'][j][i] for i in range(len(test_n['annotation'][j])) if test_n['annotation'][j][i][0] != '']
    test_n['loc'][j] = [test_n['loc'][j][i] for i in range(len(test_n['loc'][j])) if test_n['loc'][j][i][0] != '']
    start = np.array([int(test_n['loc'][j][i][0]) for i in range(len(test_n['loc'][j]))])
    order = start.argsort()
    test_n['annotation'][j] = [test_n['annotation'][j][i] for i in order]
    test_n['loc'][j] = [test_n['loc'][j][i] for i in order]
test = test_n.drop('loc', axis=1)
test.shape[0]

### modify text(eg. remove stopwords, changing coronary artery disease to CAD)

In [ ]:
'''
import nltk
from nltk.corpus import stopwords
sw = stopwords.words('english')
negat = ["no", "nor", "not","don't","didn't","doesn't","isn't","aren't","wasn't","weren't","haven't","hasn't","hadn't","won't","wouldn't","shouldn't","can't","couldn't","mustn","mustn't","mightn't","mightn't","needn't","needn't","oughtn't","shan't","shan't","shouldn't","wasn't","weren't","won't","wouldn't","t","shouldn","wasn","weren","won","wouldn","can","couldn","didn","doesn","hadn","hasn","haven","isn","mightn","mustn","needn","oughtn","shan","shouldn","wasn","weren","won","wouldn"]
sw_n = [w for w in sw if w not in negat]
'''

In [ ]:
'''
# remove stop words in text to match tags
for i in range(train.shape[0]):
    word_tokens = train['text'][i].strip().split()
    filtered_sentence = [w for w in word_tokens if not w.lower() in sw_n]
    train['text'][i] = ' '.join(filtered_sentence)'''

In [ ]:
'''
train['text'] = train['text'].apply(lambda x: re.sub('coronary artery disease','CAD',x))
train['text'] = train['text'].apply(lambda x: re.sub('Coronary artery disease','CAD',x))
train['text'] = train['text'].apply(lambda x: re.sub('Coronary Artery Disease','CAD',x))
train['text'] = train['text'].apply(lambda x: re.sub('Blood Pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('blood pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('Blood pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('blood Pressure','BP',x))
train['text'] = train['text'].apply(lambda x: re.sub('&#8211','',x))
#train['text'] = train['text'].apply(lambda x: re.sub(' p\.o\. ',' per oral ',x))
#train['text'] = train['text'].apply(lambda x: re.sub(' h/o ',' had ',x))
#train['text'] = train['text'].apply(lambda x: x.lower())'''

In [ ]:
'''
for i in range(train.shape[0]):
    for x in range(len(train['annotation'][i])):
        word_tokens = train['annotation'][i][x][0].strip().split()
        # converts the words in word_tokens to lower case and then checks whether 
        #they are present in stop_words or not
        
        filtered_sentence = [w for w in word_tokens if not w.lower() in sw_n] 
        tagged_things = ' '.join(filtered_sentence) 

        tagged_things = re.sub('coronary artery disease','CAD',tagged_things) 
        tagged_things = re.sub('Coronary artery disease','CAD',tagged_things)
        tagged_things = re.sub('Coronary Artery Disease','CAD',tagged_things)
        tagged_things = re.sub('Blood Pressure','BP',tagged_things)
        tagged_things = re.sub('blood pressure','BP',tagged_things)
        tagged_things = re.sub('Blood pressure','BP',tagged_things)
        tagged_things = re.sub('blood Pressure','BP',tagged_things)
        tagged_things = re.sub('&#8211','',tagged_things)
        #tagged_things = re.sub(' p\.o\. ',' per oral ',tagged_things)
        #tagged_things = re.sub(' h/o ',' had ',tagged_things)
        train['annotation'][i][x] = (tagged_things,train['annotation'][i][x][1])'''

### update info for corpus

In [ ]:
## update info in corpus
corpus['CleanedText'] = train['text']
corpus['AnnotatedText'] = train['annotation']

#Tag categories
tagCategories = []
for index,entry in enumerate(corpus['AnnotatedText']):
    tagcatergories_eachfile = []
    for entity in entry:
        tagcatergories_eachfile.append(entity[1])
    tagCategories.append(tagcatergories_eachfile)

corpus['TagCategory'] = tagCategories
tag_doc = []
for index,entry in enumerate(corpus['TagCategory']):
    tag_doc_eachfile = []
    for i in range(len(entry)):
        #just keep the unique tag category, store in doc-level tag
        tag_doc_eachfile = list(set(entry))
    tag_doc.append(tag_doc_eachfile)

corpus['Tag_doc'] = tag_doc

#save the corpus to a csv file
corpus.to_csv('E:/JHU/课程/datadesign/NLP/machine_learning/corpus.csv',index=False,encoding='utf-8')

In [ ]:
corpus_test['CleanedText'] = test['text']
corpus_test['AnnotatedText'] = test['annotation']

#Tag categories
tagCategories = []
for index,entry in enumerate(corpus_test['AnnotatedText']):
    tagcatergories_eachfile = []
    for entity in entry:
        tagcatergories_eachfile.append(entity[1])
    tagCategories.append(tagcatergories_eachfile)

corpus_test['TagCategory'] = tagCategories
tag_doc = []
for index,entry in enumerate(corpus_test['TagCategory']):
    tag_doc_eachfile = []
    for i in range(len(entry)):
        #just keep the unique tag category, store in doc-level tag
        tag_doc_eachfile = list(set(entry))
    tag_doc.append(tag_doc_eachfile)

corpus_test['Real_tag_doc'] = tag_doc

#save the corpus to a csv file
corpus_test.to_csv('E:/JHU/课程/datadesign/NLP/machine_learning/corpus_test.csv',index=False,encoding='utf-8')

--------------------------------------------------------------------------------------Next section-----------------------------------------------------------------------------------------

# Create input for machine learning

In [ ]:
from tqdm import tqdm
from difflib import SequenceMatcher
import pickle

## define function: get location to tag in text
## string: original text   pattern: tagged text
def matcher(string, pattern):
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        ## replace matcheed tags with X to avoid double match
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)

    return match_list, string

## create word by word tag input file
## s: original text   match_list: output from matcher()
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = e_type
        else:
            word_dict.loc[index,'label'] = e_type
    return word_dict

#def clean(text):
#    '''
#    Just a helper fuction to add a space before the punctuations for better tokenization
#    '''
#    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
#               "\\", "]", "_", "`", "{", "}", "~", "'"]
#    for i in text:
#        if i in filters:
#            text = text.replace(i, " " + i)
#            
#    return text

## integrate matcher)() and create_labs()
def to_txt(df, corpus, filepath):
    with open(filepath , 'w') as f:
        n = 0
        for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            text_ = text    
            print(df.index[df['text']== text_].tolist())    
            match_list = []
            for i in annotation:
                a,text_= matcher(text_, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text, match_list)

            word_label = []
            for i in range(d.shape[0]):
                #f.writelines(d['word'][i] + ' ' + d['label'][i] +'\n')
                word_label.append(d['word'][i] + ' ' + d['label'][i])

            corpus['Real_text_tag'][n] = word_label
            n += 1
           
def main(input,corpus,save_path):

    data = input
    to_txt(data,corpus, save_path)


In [ ]:
path = './'
main(train,corpus,path+'train(clean).txt')
main(test,corpus_test,path+'test(clean).txt')

In [ ]:
#save the corpus to a csv file
corpus.to_csv('E:/JHU/课程/datadesign/NLP/machine_learning/corpus.csv',index=False,encoding='utf-8')
corpus_test.to_csv('E:/JHU/课程/datadesign/NLP/machine_learning/corpus_test.csv',index=False,encoding='utf-8')

In [ ]:
## may need to load corpus

traininglist = pd.DataFrame(columns=['file','text', 'label'])
for index,entry in enumerate(corpus['Real_text_tag']):
    trainingsublist = pd.DataFrame(np.zeros((len(entry), 3), dtype=object), columns=['file','text', 'label'])    
    #make the text and label into two columns, and then concat them together
    for i in range(len(entry)):
        trainingsublist['text'][i] = entry[i].split()[0]
        trainingsublist['label'][i] = entry[i].split()[1]
    trainingsublist['file'] = index+1
    traininglist = pd.concat([traininglist,trainingsublist],axis=0)


    testinglist = pd.DataFrame(columns=['file','text', 'label'])
for index,entry in enumerate(corpus_test['Real_text_tag']):
    testingsublist = pd.DataFrame(np.zeros((len(entry), 3), dtype=object), columns=['file','text', 'label'])
    #make the text and label into two columns, and then concat them together
    for i in range(len(entry)):
        testingsublist['text'][i] = entry[i].split()[0]
        testingsublist['label'][i] = entry[i].split()[1]
    testingsublist['file'] = index+1
    testinglist = pd.concat([testinglist,testingsublist],axis=0)

### modify text(eg. remove stopwords, changing coronary artery disease to CAD)

In [ ]:
'''
for i in range(test.shape[0]):
    word_tokens = test['text'][i].strip().split()
    filtered_sentence = [w for w in word_tokens if not w.lower() in sw_n]
    test['text'][i] = ' '.join(filtered_sentence)'''

In [ ]:
'''
test['text'] = test['text'].apply(lambda x: re.sub('coronary artery disease','CAD',x))
test['text'] = test['text'].apply(lambda x: re.sub('Coronary artery disease','CAD',x))
test['text'] = test['text'].apply(lambda x: re.sub('Coronary Artery Disease','CAD',x))
test['text'] = test['text'].apply(lambda x: re.sub('Blood Pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('blood pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('Blood pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('blood Pressure','BP',x))
test['text'] = test['text'].apply(lambda x: re.sub('&#8211','',x))
#test['text'] = test['text'].apply(lambda x: re.sub(' p\.o\. ','per oral',x))
#test['text'] = test['text'].apply(lambda x: re.sub(' h/o ','had',x))
#test['text'] = test['text'].apply(lambda x: x.lower())'''

In [ ]:
'''
for i in range(test.shape[0]):
    for x in range(len(test['annotation'][i])):
        word_tokens = test['annotation'][i][x][0].strip().split()
        # converts the words in word_tokens to lower case and then checks whether 
        #they are present in stop_words or not
        filtered_sentence = [w for w in word_tokens if not w.lower() in sw_n]
        tagged_things = ' '.join(filtered_sentence)
        tagged_things = re.sub('coronary artery disease','CAD',tagged_things)
        tagged_things = re.sub('Coronary artery disease','CAD',tagged_things)
        tagged_things = re.sub('Coronary Artery Disease','CAD',tagged_things)
        tagged_things = re.sub('Blood Pressure','BP',tagged_things)
        tagged_things = re.sub('blood pressure','BP',tagged_things)
        tagged_things = re.sub('Blood pressure','BP',tagged_things)
        tagged_things = re.sub('blood Pressure','BP',tagged_things)
        tagged_things = re.sub('&#8211','',tagged_things)
        #tagged_things = re.sub(' p\.o\. ',' per oral ',tagged_things)
        #tagged_things = re.sub(' h/o ','had',tagged_things)
        test['annotation'][i][x] = (tagged_things,test['annotation'][i][x][1])'''

--------------------------------------------------------------------------------------Next section-----------------------------------------------------------------------------------------

# Modelling (Naive Bayes)

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

In [ ]:
## load corpus if already have one

traininglist = pd.DataFrame(columns=['file','text', 'label'])
for index,entry in enumerate(corpus['Real_text_tag']):
    trainingsublist = pd.DataFrame(np.zeros((len(entry), 3), dtype=object), columns=['file','text', 'label'])    
    #make the text and label into two columns, and then concat them together
    for i in range(len(entry)):
        trainingsublist['text'][i] = entry[i].split()[0]
        trainingsublist['label'][i] = entry[i].split()[1]
    trainingsublist['file'] = index+1
    traininglist = pd.concat([traininglist,trainingsublist],axis=0)


    testinglist = pd.DataFrame(columns=['file','text', 'label'])
for index,entry in enumerate(corpus_test['Real_text_tag']):
    testingsublist = pd.DataFrame(np.zeros((len(entry), 3), dtype=object), columns=['file','text', 'label'])
    #make the text and label into two columns, and then concat them together
    for i in range(len(entry)):
        testingsublist['text'][i] = entry[i].split()[0]
        testingsublist['label'][i] = entry[i].split()[1]
    testingsublist['file'] = index+1
    testinglist = pd.concat([testinglist,testingsublist],axis=0)

In [ ]:
trainingtext = traininglist.drop(['file','label'],axis=1)
#print(trainingtext)
traininglabel = traininglist['label']

v = DictVectorizer(sparse=True)
x_train = v.fit_transform(trainingtext.to_dict('records'))
y_train = traininglabel

classes = np.unique(traininglabel)
classes = classes.tolist()

testingtext = testinglist.drop(['file','label'],axis=1)
testinglabel = testinglist['label']

x_test = v.transform(testingtext.to_dict('records'))
y_test = testinglabel

x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
new_classes = classes.copy()
## O class bias evaluation, remove
new_classes.remove('O')

In [ ]:
#Naive bayes
nb = MultinomialNB(alpha=.0005)
nb.partial_fit(x_train, y_train,classes=classes)

print (classification_report(y_pred=nb.predict(x_test), y_true=y_test, labels=new_classes))

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

y_pred = nb.predict(x_test)
conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
fig, ax = plt.subplots(figsize=(10,8))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=classes, yticklabels=classes)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

## Prediction 

In [ ]:
vadicationlist = testinglist.copy()
vadicationlist.insert(3, 'pred_label', '')

vadicationlist['pred_label'] = y_pred

vadicationlist.groupby('pred_label').count().sort_values(by='text',ascending=False)

In [ ]:
print(vadicationlist.groupby(['label']).count().sort_values(by='text',ascending=False).drop(['file','pred_label'],axis=1))
print(vadicationlist.groupby(['pred_label']).count().sort_values(by='text',ascending=False).drop(['file','label'],axis=1))

In [ ]:
#get the different result
wrongresult = vadicationlist[vadicationlist['label']!=vadicationlist['pred_label']].copy()
print(wrongresult)
sorted_wrong_result = wrongresult.groupby('text').count().sort_values(by='label', ascending=False) #sort the wrong result by the number of times it appears
print(sorted_wrong_result[:15])


In [ ]:
print(wrongresult[wrongresult['text'] == 'hypertension,'])

In [ ]:
vadicationlist['file']-1

In [ ]:
#group the vadicationlist by file
vadicationlist_grouped = vadicationlist.groupby('file')
vadicationlist_grouped_list = []
for name,group in vadicationlist_grouped:
    vadicationlist_grouped_list.append(group)

print(vadicationlist_grouped_list[0])

#delete the 'file' column
for i in range(len(vadicationlist_grouped_list)):
    vadicationlist_grouped_list[i] = vadicationlist_grouped_list[i].drop(['file'],axis=1)


In [ ]:
print(vadicationlist_grouped_list)

In [ ]:
vadicationlist_grouped_list[0].shape

In [ ]:
Pred_text_tag = []
for i in range(len(vadicationlist_grouped_list)):
    Pred_text_tag_eachfile = []
    for j in range(len(vadicationlist_grouped_list[i])):
        Pred_text_tag_eachfile.append(vadicationlist_grouped_list[i]['text'][j]+' '+vadicationlist_grouped_list[i]['label'][j]+' '+vadicationlist_grouped_list[i]['pred_label'][j])
    Pred_text_tag.append(Pred_text_tag_eachfile)

In [ ]:
print(len(Pred_text_tag[0]))
print(len(Pred_text_tag[1]))

In [ ]:
corpus_test['Pred_text_tag'] = Pred_text_tag

In [ ]:
corpus_test['Pred_text_tag'][0]

In [ ]:
#save the corpus to a csv file
corpus_test.to_csv('E:/JHU/课程/datadesign/NLP/machine_learning/corpus_test.csv',index=False,encoding='utf-8')

# Doc-level Prediction

In [ ]:
Pred_doc_tag = []
#delete repeated tags
for i in range(len(vadicationlist_grouped_list)):
    Pred_doc_tag_eachfile = []
    for j in range(len(vadicationlist_grouped_list[i])):
        Pred_doc_tag_eachfile.append(vadicationlist_grouped_list[i]['pred_label'][j])
    Pred_doc_tag_eachfile = list(set(Pred_doc_tag_eachfile))
    Pred_doc_tag_eachfile.remove('O')
    Pred_doc_tag.append(Pred_doc_tag_eachfile)
#vadicationlist_grouped_list[0]['pred_label']

In [ ]:
print(Pred_doc_tag[2])

In [ ]:
corpus_test['Pred_tag_doc'] = Pred_doc_tag

In [ ]:
#save the corpus to a csv file
corpus_test.to_csv('E:/JHU/课程/datadesign/NLP/machine_learning/corpus_test.csv',index=False,encoding='utf-8')

## Rule-based doc-level prediction for SMOKER

In [ ]:
file2tag = {}
import os

with open('./TestingFAMILY_HIST.txt','r') as f:
    for line in f:
        lineInfo = line.strip().split(' ')
        if len(lineInfo) > 1:
            fileName = lineInfo[1]
            if fileName in file2tag: file2tag[fileName].append(lineInfo[2:])
            else: file2tag[fileName] = [lineInfo[2:]]

SMOKER_path = './Smoker/'

for fileName in os.listdir(testingpath): #Input EHRs
    with open(testingpath+'/'+fileName,'r') as f:
        with open(SMOKER_path+fileName,'w') as fw: #Can change this to any folder name you want (this is where it puts the outputs)
            f.readline()
            f.readline()
            fileString = ''
            tags = False
            tagsList = []
            doccount = 0
            fw.write("<?xml version='1.0' encoding='UTF-8'?>\n")
            fw.write("<root>\n")
            for line in f.readlines():
                fileString += line
                if tags: tagsList.append(line)
                else: fw.write(line)
                if '<TAGS>' in line: tags = True
            histKnown = False
            for tag in file2tag[fileName]:
                typeOfTag = tag[-1].strip().replace('\n','').replace('\t','')
                if typeOfTag == 'familyhistory':
                    histKnown = True
            if not histKnown:
                fw.write("\t<FAMILY_HIST id=\"DOC"+str(doccount)+"\" indicator=\"not present\"/>\n")
                doccount += 1
            else:
                fw.write("\t<FAMILY_HIST id=\"DOC"+str(doccount)+"\" indicator=\"present\"/>\n")
                doccount += 1
            fw.write('\t</TAGS>\n')
            fw.write('</root>')

In [ ]:
SMOKER_prediction = []

for file in os.listdir(SMOKER_path):
    tree = ET.parse(SMOKER_path+file)
    root = tree.getroot()
    tags = root.findall('TAGS')
    for tag in tags:
        for child in tag:
            if child.tag == 'FAMILY_HIST':
                #in family history, the indicator is either present or not present, store it in the list
                SMOKER_prediction.append(child.attrib['indicator']) 
                

print(len(SMOKER_prediction))


In [ ]:
#count the number of 'present'
print(SMOKER_prediction.count('present'))

In [ ]:
for i in range(len(SMOKER_prediction)):
    if SMOKER_prediction[i] == 'present':
        #if the indicator is present, add the tag 'FAMILY_HIST' to the the doc-level tag list
        Pred_doc_tag[i].append('FAMILY_HIST')

In [ ]:
#Count the number of 'FAMILY_HIST' in the doc-level tag list
count = 0
for i in range(len(Pred_doc_tag)):
    if 'FAMILY_HIST' in Pred_doc_tag[i]:
        count += 1
print(count)

In [ ]:
corpus_test['Pred_tag_doc'] = Pred_doc_tag #update the doc-level tag list

In [ ]:
#save the corpus to a csv file
corpus_test.to_csv('E:/JHU/课程/datadesign/NLP/machine_learning/corpus_test.csv',index=False,encoding='utf-8')

# Evaluation

In [ ]:
Evaluation = pd.DataFrame(columns=['file','Real_tag_doc','Pred_tag_doc'])
Evaluation['file'] = corpus_test.index
Evaluation['Real_tag_doc'] = corpus_test['Real_tag_doc']
Evaluation['Pred_tag_doc'] = Pred_doc_tag

Evaluation

In [ ]:
Evaluation['Real_tag_doc'][0]

In [ ]:
def evaluate_each(list, taget_tag):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for i in range(len(list)):
        if taget_tag in list['Real_tag_doc'][i]:
            if taget_tag in list['Pred_tag_doc'][i]:
                TP += 1
            else:
                FN += 1
        else:
            if taget_tag in list['Pred_tag_doc'][i]:
                FP += 1
            else:
                TN += 1
            
    #print('TP:',TP)
    #print('FP:',FP)
    #print('FN:',FN)
    return TP,FP,FN,TN

def evaluate_matrix(list, taget_tag_list = ['SMOKER', 'FAMILY_HIST']):
    
    score_matrix = pd.DataFrame(columns=['target_tag','TP','FP','FN','TN','precision','recall','F1','Support'])

    for taget_tag in taget_tag_list:
        score_matrix_each = pd.DataFrame(columns=['target_tag','TP','FP','FN','TN','precision','recall','F1','Support'])
        TP,FP,FN,TN = evaluate_each(list, taget_tag)
        if TP+FP == 0:
            if TP == 0:
                precision = 0
            else:
                precision = 1
        else:
            precision = TP/(TP+FP)
        if TP+FN == 0:
            if TP == 0:
                recall = 0
            else:
                recall = 1
        else:
            recall = TP/(TP+FN)
        if precision+recall == 0:
            if precision*recall == 0:
                F1 = 0
            else:
                F1 = 1
        else:
            F1 = 2*precision*recall/(precision+recall)
        
        score_matrix_each['target_tag'] = [taget_tag]
        score_matrix_each['TP'] = [TP]
        score_matrix_each['FP'] = [FP]
        score_matrix_each['FN'] = [FN]
        score_matrix_each['TN'] = [TN]
        score_matrix_each['precision'] = [precision]
        score_matrix_each['recall'] = [recall]
        score_matrix_each['F1'] = [F1]
        score_matrix_each['Support'] = [TP+FN]

        
        score_matrix = pd.concat([score_matrix,score_matrix_each],axis=0)
        
    return score_matrix

def total_score(score_matrix):  
    support_sum = score_matrix['Support'].sum()
    support_proportion = score_matrix['Support']/support_sum

    TP_sum = score_matrix['TP'].sum()
    FP_sum = score_matrix['FP'].sum()
    FN_sum = score_matrix['FN'].sum()
    TN_sum = score_matrix['TN'].sum()
    
    weighted_precision = (score_matrix['precision']*support_proportion).sum()
    weighted_recall = (score_matrix['recall']*support_proportion).sum()
    weighted_F1 = (score_matrix['F1']*support_proportion).sum()

    
    score_matrix = score_matrix.append({'target_tag':'Total_macro','TP':TP_sum,'FP':FP_sum,'FN':FN_sum, 'TN':TN_sum,
                                        'precision':score_matrix['precision'].mean(),'recall':score_matrix['recall'].mean(),'F1':score_matrix['F1'].mean(), 'Support':support_sum},ignore_index=True)
    score_matrix = score_matrix.append({'target_tag':'Total_micro','TP':TP_sum,'FP':FP_sum,'FN':FN_sum, 'TN':TN_sum,
                                        'precision':score_matrix['TP'].sum()/(score_matrix['TP'].sum()+score_matrix['FP'].sum()),'recall':score_matrix['TP'].sum()/(score_matrix['TP'].sum()+score_matrix['FN'].sum()),'F1':2*score_matrix['TP'].sum()/(2*score_matrix['TP'].sum()+score_matrix['FP'].sum()+score_matrix['FN'].sum()), 'Support':support_sum},ignore_index=True)
    score_matrix = score_matrix.append({'target_tag':'Total_weighted','TP':TP_sum,'FP':FP_sum,'FN':FN_sum, 'TN':TN_sum,
                                        'precision':weighted_precision,'recall':weighted_recall,'F1':weighted_F1, 'Support':support_sum},ignore_index=True)
    
    #print(score_matrix)
    return score_matrix



In [ ]:
Evaluate_Matrix = evaluate_matrix(Evaluation,taget_tag_list =['MEDICATION','CAD','DIABETES','SMOKER','HYPERTENSION','HYPERLIPIDEMIA','OBESE','FAMILY_HIST'])

#no smoker
#Evaluate_Matrix = evaluate_matrix(Evaluation,taget_tag_list =['MEDICATION','CAD','DIABETES','HYPERTENSION','HYPERLIPIDEMIA','OBESE','FAMILY_HIST'])
Evaluate_Matrix

In [ ]:
total_score(Evaluate_Matrix)

In [ ]:
#plot the precision recall F1 
import matplotlib.pyplot as plt
import numpy as np

def plot_score(score_matrix):
    #sco
    score_matrix = score_matrix.sort_values(by=['F1'],ascending=False)
    score_matrix = score_matrix.reset_index(drop=True)
    score_matrix = score_matrix.drop(['TP','FP','FN','TN','Support'],axis=1)
    score_matrix = score_matrix.set_index('target_tag')
    score_matrix.plot(kind='barh',figsize=(6,6),fontsize=9, colormap='Paired')
    plt.show()

plot_score(Evaluate_Matrix)





In [ ]:
#accuracy
accuracy = (Evaluate_Matrix['TP'].sum()+Evaluate_Matrix['TN'].sum())/(Evaluate_Matrix['TP'].sum()+Evaluate_Matrix['TN'].sum()+Evaluate_Matrix['FP'].sum()+Evaluate_Matrix['FN'].sum())
print('accuracy:',accuracy)